In [1]:
import pandas as pd     
import plotly           
import plotly.express as px
import dash             
from dash import dash_table
import pandas as pd 
import numpy as np
import plotly.express as px
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc

In [2]:
df = pd.read_excel("COVID-19-geographic-disbtribution-worldwide-2020-03-29.xlsx")

dff = df.groupby('countriesAndTerritories', as_index=False)[['deaths','cases']].sum()

In [3]:
dff.head()

,countriesAndTerritories,deaths,cases
0,Afghanistan,3,106
1,Albania,10,197
2,Algeria,26,409
3,Andorra,4,308
4,Angola,0,4


In [6]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dbc.Row([  #1.row row kullandıktqan sonra kolon kullan, birden fazla column kullanılabilir ama burada direkt title olduğu için bütün rowu kapsar 
        dbc.Col(
            dash_table.DataTable(
            id='datatable_id',
            data=dff.to_dict('records'),
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": False} for i in dff.columns
            ],
            editable=False,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            row_selectable="multi",
            row_deletable=False,
            selected_rows=[],
            page_action="native",
            page_current= 0,
            page_size= 6,
            # page_action='none',
            # style_cell={
            # 'whiteSpace': 'normal'
            # },
            # fixed_rows={ 'headers': True, 'data': 0 },
            # virtualization=False,
            style_cell_conditional=[
                                    {'if': {'column_id': 'countriesAndTerritories'},
                                     'width': '40%', 'textAlign': 'left'},
                                    {'if': {'column_id': 'deaths'},
                                     'width': '30%', 'textAlign': 'left'},
                                    {'if': {'column_id': 'cases'},
                                     'width': '30%', 'textAlign': 'left'},
                                    ],
            ),
            width=12
        )
    ]),
    
    dbc.Row([  
        dbc.Col(
            dcc.Dropdown(id='linedropdown',
                            options=[
                             {'label': 'Deaths', 'value': 'deaths'},
                             {'label': 'Cases', 'value': 'cases'}
                                ],
                            value='deaths',
                            multi=False,
                            clearable=False
                            ),
                width=6
                ),
        
        dbc.Col(
            dcc.Dropdown(id='piedropdown',
                        options=[{'label': 'Deaths', 'value': 'deaths'},
                                 {'label': 'Cases', 'value': 'cases'}],
                        value='cases',
                        multi=False,
                        clearable=False
                        ),
                width=6
                )
    ]),
    
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='linechart'),
            ], 
            width={'size':6}
            ),
        
        dbc.Col([
            dcc.Graph(id='piechart'),
            ], 
            width={'size':6}
            ),
        
        ])
], fluid=True #default fluid false ancak bu rowlarda space bırakır sağda ve solda başlangıç bitiş boşluk koyar bunun için width kullanılabilir offset ile 
)


@app.callback(
            [Output('piechart','figure'),
             Output('linechart','figure')],
            [Input('datatable_id','selected_rows'),
             Input('piedropdown','value'),
             Input('linedropdown','value')]
)

def update_data(chosen_rows,piedropval,linedropval):
    if len(chosen_rows)==0:
        df_filtered = dff[dff["countriesAndTerritories"].isin(["China","Iran","Spain","Italy"])]
    else:
        #print(chosen_rows)
        df_filtered= dff[dff.index.isin(chosen_rows)]
        
    pie_chart = px.pie(data_frame=df_filtered,
                        names='countriesAndTerritories',
                        values=piedropval,
                        hole=0.3,
                        labels={"countriesAndTerritories":"Countries"}
                        )
    
    
    list_chosen_countries = df_filtered["countriesAndTerritories"].tolist()
    df_line = df[df['countriesAndTerritories'].isin(list_chosen_countries)]
    line_chart = px.line(
                        data_frame=df_line,
                        x='dateRep',
                        y=linedropval,
                        color='countriesAndTerritories',
                        labels={'countriesAndTerritories':'Countries', 'dateRep':'date'}
                        )
    
    line_chart.update_layout(uirevision='foo')
    return (pie_chart,line_chart)

if __name__ == "__main__":
    app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

